In [ ]:
import pickle
import numpy as np
import networkx as nx

# Step 1: Load the binary adjacency matrix from the pkl file
file_path = r''
with open(file_path, 'rb') as f:
    matrices_list = pickle.load(f)

# Step 2: Convert the binary adjacency matrix to a NetworkX graph
adjacency_matrix = matrices_list[i]  # Selecting one i random matrix from the list
G = nx.from_numpy_array(adjacency_matrix)

# Step 3: Calculate betweenness centrality for each node
betweenness_centralities = nx.betweenness_centrality(G)

# Step 4: Assign total population (P) to the network
total_population = p(i)  # Set the total population based on your criteria

# Step 5: Distribute the total population among nodes using betweenness centrality as weights
population_per_node = {}
for node, betweenness_centrality in betweenness_centralities.items():
    population_per_node[node] = int(betweenness_centrality * total_population)

# Step 6: Distribute trips originated at each node to other nodes
# Calculate the shortest path lengths between all pairs of nodes
shortest_path_lengths = dict(nx.shortest_path_length(G))

# Function to calculate distance between nodes based on shortest path lengths or any other impedance function
#def distance_between_nodes(node_i, node_j):
def impedance(node_i, node_j):
    if node_i == node_j:
        return 0
    shortest_path_len = shortest_path_lengths[node_i][node_j]
    return 1 / (shortest_path_len ** 2)

# Generate demand between all nodes based on their population and distance
num_nodes = G.number_of_nodes()
demand_matrix = np.zeros((num_nodes, num_nodes), dtype=float)

for node_i in G.nodes():
    for node_j in G.nodes():
        if node_i == node_j:
            continue
        demand_matrix[node_i][node_j] = (
            #population_per_node[node_i] * distance_between_nodes(node_i, node_j)
            population_per_node[node_i] * impedance(node_i, node_j)
        )

# Now, 'demand_matrix' represents the traffic demand between all pairs of nodes in the graph.
# You can use this demand matrix for further analysis or simulations.

# Optionally, if you want to visualize the generated graph and the demand matrix, you can use:
import matplotlib.pyplot as plt

# Plot the graph
pos = nx.spring_layout(G)  # You can choose other layout algorithms as well
nx.draw(G, pos, with_labels=True)
plt.show()

# Plot the demand matrix as a heatmap
plt.imshow(demand_matrix, cmap='hot', interpolation='nearest')
plt.colorbar()
plt.show()

import pandas as pd
# Convert the demand matrix to a pandas DataFrame
demand_df = pd.DataFrame(demand_matrix, columns=G.nodes(), index=G.nodes())

# Save the demand DataFrame as a CSV file
output_csv_path = r''
demand_df.to_csv(output_csv_path)

# Optionally, you can also save the DataFrame as a pickle file (.pkl)
output_pkl_path = r''
demand_df.to_pickle(output_pkl_path)